Querying data that will be used to train the neural network
---
1) querying all coins for which will build a correlated neural network model

2) keeping only the data of interest in an array:

    t-m:   p1 p2 p3 p4 ... pn
    t-m+1: p1 p2 p3 p4 ... pn
    t-m+2: p1 p2 p3 p4 ... pn 
    ...
    t:     p1 p2 p3 p4 ... pn

3) the vars array will have shape (n currencies, m points)

In [1]:
import numpy as np

from cryptocompare import get_histo

"""
hour - 2000 points max
minute - 2000 points max
"""

vars = []
mhd = 'hour'
candle_width = 1
m = 2000
e = 'CCCAGG'

# Selected cryptos with higher market caps (https://coinmarketcap.com/)
currencies = ['BTC', 'ETH', 'XRP', 'BCH', 'LTC', 'NEO', 'XLM']
n = len(currencies)
for coin in currencies:
    q = get_histo(mhd, coin, m, candle_width, e)
    data = q['Data']
    var = []
    for d0, d1 in zip(data[:-1], data[1:]):
        if d0['close'] == 0:
            raise ValueError('Invalid input for crypto %s: %s' % (coin, str(d0)))
        else:
            p = d1['close']/d0['close'] - 1
        var.append(p)
    vars.append(var)
vars = np.array(vars)
print('vars.shape', vars.shape)

vars.shape (7, 2000)


### Particioning blocks of data that will be used to build the neural network
---
1) determining the number of points for each block

2) building vector of inputs (xs) and outputs (ys)

3) classifying responses for each cryptocurrency using the following scheme (only 2 classes, I tried with more classes but the prediction gets much worse):

    0 -->     y < threshold
    1 -->     y >= threshold
    

In [2]:
block = 50
xs = []
ys = []
for i in range(vars.shape[1] - (block+1)):
    part = vars[:, i:i+(block+1)]
    xs.append(part[:, :-1])
    ys.append(part[:, -1])
xs = np.array(xs)
print('xs.shape', xs.shape)
def classify(y):
    threshold = 0.02
    if y < threshold:
        return 0
    else:
        return 1

classes = {}
for i, currency in enumerate(currencies):
    tmp = []
    for y in ys:
        tmp.append(classify(y[i]))
    classes[currency] = np.array(tmp)
    

xs.shape (1949, 7, 50)


Shuffling samples for a better stochastic response
---

In [3]:
import random
rand_indices = np.arange(xs.shape[0])
random.shuffle(rand_indices)
xs = xs[rand_indices]
xs = xs.reshape(xs.shape[0], -1)
print('xs.shape', xs.shape)
for k in classes:
    classes[k] = classes[k][rand_indices]

xs.shape (1949, 350)


Selecting training set and testing set
---

In [4]:
train_size = 1700

xs_train = xs[:train_size]
xs_test = xs[train_size:]
classes_train = {}
classes_test = {}
for k in classes:
    classes_train[k] = classes[k][:train_size]
    classes_test[k] = classes[k][train_size:]

Training a simple Keras model
---

Reference: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [5]:
from keras.models import Sequential
from keras.layers import Dense

models = {}
for k in currencies:
    Y = classes_train[k]
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=xs_train.shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    out = model.fit(xs_train, Y, epochs=250, batch_size=0)
    models[k] = model

Using TensorFlow backend.


Epoch 1/250
1700/1700 [==============================] - 0s 199us/step - loss: 0.6843 - acc: 0.7088
Epoch 2/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.6442 - acc: 0.9053
Epoch 3/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.5985 - acc: 0.9047
Epoch 4/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.5464 - acc: 0.9047
Epoch 5/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.4909 - acc: 0.9047
Epoch 6/250
1700/1700 [==============================] - 0s 27us/step - loss: 0.4300 - acc: 0.9047
Epoch 7/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.3670 - acc: 0.9047
Epoch 8/250
1700/1700 [==============================] - 0s 27us/step - loss: 0.3214 - acc: 0.9047
Epoch 9/250
1700/1700 [==============================] - 0s 27us/step - loss: 0.2981 - acc: 0.9047
Epoch 10/250
1700/1700 [==============================] - 0s 27us/step - loss: 0.2860 - acc: 0.9053
Epoch 11

1700/1700 [==============================] - 0s 22us/step - loss: 0.0364 - acc: 0.9900
Epoch 84/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.0354 - acc: 0.9906
Epoch 85/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.0342 - acc: 0.9906
Epoch 86/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0334 - acc: 0.9912
Epoch 87/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.0324 - acc: 0.9912
Epoch 88/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0316 - acc: 0.9912
Epoch 89/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0308 - acc: 0.9912
Epoch 90/250
1700/1700 [==============================] - 0s 20us/step - loss: 0.0300 - acc: 0.9912
Epoch 91/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0293 - acc: 0.9912
Epoch 92/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0284 - acc: 0.9912
Epoch 93/250


1700/1700 [==============================] - 0s 21us/step - loss: 0.0023 - acc: 1.0000
Epoch 165/250
1700/1700 [==============================] - 0s 26us/step - loss: 0.0023 - acc: 1.0000
Epoch 166/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0022 - acc: 1.0000
Epoch 167/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0021 - acc: 1.0000
Epoch 168/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.0020 - acc: 1.0000
Epoch 169/250
1700/1700 [==============================] - 0s 20us/step - loss: 0.0020 - acc: 1.0000
Epoch 170/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0019 - acc: 1.0000
Epoch 171/250
1700/1700 [==============================] - 0s 20us/step - loss: 0.0018 - acc: 1.0000
Epoch 172/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0018 - acc: 1.0000
Epoch 173/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.0017 - acc: 1.0000
Epoc

1700/1700 [==============================] - 0s 28us/step - loss: 2.4491e-04 - acc: 1.0000
Epoch 244/250
1700/1700 [==============================] - 0s 23us/step - loss: 2.3745e-04 - acc: 1.0000
Epoch 245/250
1700/1700 [==============================] - 0s 23us/step - loss: 2.3416e-04 - acc: 1.0000
Epoch 246/250
1700/1700 [==============================] - 0s 21us/step - loss: 2.2719e-04 - acc: 1.0000
Epoch 247/250
1700/1700 [==============================] - 0s 22us/step - loss: 2.2135e-04 - acc: 1.0000
Epoch 248/250
1700/1700 [==============================] - 0s 22us/step - loss: 2.1598e-04 - acc: 1.0000
Epoch 249/250
1700/1700 [==============================] - 0s 21us/step - loss: 2.1193e-04 - acc: 1.0000
Epoch 250/250
1700/1700 [==============================] - 0s 22us/step - loss: 2.0642e-04 - acc: 1.0000
Epoch 1/250
1700/1700 [==============================] - 0s 264us/step - loss: 0.6636 - acc: 0.8376
Epoch 2/250
1700/1700 [==============================] - 0s 24us/step - lo

...

1700/1700 [==============================] - 0s 21us/step - loss: 0.0201 - acc: 0.9988
Epoch 120/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.0194 - acc: 0.9982
Epoch 121/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0186 - acc: 0.9982
Epoch 122/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0178 - acc: 0.9988
Epoch 123/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0173 - acc: 0.9988
Epoch 124/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.0166 - acc: 0.9988
Epoch 125/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.0159 - acc: 0.9994
Epoch 126/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0153 - acc: 0.9994
Epoch 127/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.0145 - acc: 0.9994
Epoch 128/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0141 - acc: 0.9994
Epoc

1700/1700 [==============================] - 0s 27us/step - loss: 0.0014 - acc: 1.0000
Epoch 201/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.0013 - acc: 1.0000
Epoch 202/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0013 - acc: 1.0000
Epoch 203/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0013 - acc: 1.0000
Epoch 204/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0012 - acc: 1.0000
Epoch 205/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.0012 - acc: 1.0000
Epoch 206/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0012 - acc: 1.0000
Epoch 207/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.0011 - acc: 1.0000
Epoch 208/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.0011 - acc: 1.0000
Epoch 209/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0011 - acc: 1.0000
Epoc

1700/1700 [==============================] - 0s 24us/step - loss: 0.3483 - acc: 0.8541
Epoch 31/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.3433 - acc: 0.8576
Epoch 32/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.3374 - acc: 0.8594
Epoch 33/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.3323 - acc: 0.8612
Epoch 34/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.3276 - acc: 0.8653
Epoch 35/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.3225 - acc: 0.8676
Epoch 36/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.3178 - acc: 0.8694
Epoch 37/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.3127 - acc: 0.8753
Epoch 38/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.3078 - acc: 0.8771
Epoch 39/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.3036 - acc: 0.8788
Epoch 40/250


1700/1700 [==============================] - 0s 24us/step - loss: 0.0918 - acc: 0.9782
Epoch 112/250
1700/1700 [==============================] - 0s 21us/step - loss: 0.0898 - acc: 0.9812
Epoch 113/250
1700/1700 [==============================] - 0s 22us/step - loss: 0.0880 - acc: 0.9824
Epoch 114/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0865 - acc: 0.9835
Epoch 115/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0849 - acc: 0.9824
Epoch 116/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.0832 - acc: 0.9841
Epoch 117/250
1700/1700 [==============================] - 0s 23us/step - loss: 0.0819 - acc: 0.9847
Epoch 118/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0800 - acc: 0.9847
Epoch 119/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0789 - acc: 0.9835
Epoch 120/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.0759 - acc: 0.9847
Epoc

1700/1700 [==============================] - 0s 24us/step - loss: 0.0105 - acc: 0.9994
Epoch 193/250
1700/1700 [==============================] - 0s 29us/step - loss: 0.0103 - acc: 1.0000
Epoch 194/250
1700/1700 [==============================] - 0s 27us/step - loss: 0.0099 - acc: 1.0000
Epoch 195/250
1700/1700 [==============================] - 0s 27us/step - loss: 0.0096 - acc: 1.0000
Epoch 196/250
1700/1700 [==============================] - 0s 27us/step - loss: 0.0092 - acc: 0.9994
Epoch 197/250
1700/1700 [==============================] - 0s 26us/step - loss: 0.0091 - acc: 1.0000
Epoch 198/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.0087 - acc: 1.0000
Epoch 199/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0085 - acc: 1.0000
Epoch 200/250
1700/1700 [==============================] - 0s 24us/step - loss: 0.0082 - acc: 1.0000
Epoch 201/250
1700/1700 [==============================] - 0s 25us/step - loss: 0.0080 - acc: 1.0000
Epoc

In [6]:
xs_test.shape

(249, 350)

Evaluating the models for each currency
---

In [10]:
for k in currencies:
    scores = models[k].evaluate(xs_test, classes_test[k])
    print("Currency %s. %s: %.2f%%" % (k, models[k].metrics_names[1], scores[1]*100))    

249/249 [==============================] - 0s 32us/step
Currency BTC. acc: 83.94%
249/249 [==============================] - 0s 32us/step
Currency ETH. acc: 82.33%
249/249 [==============================] - 0s 36us/step
Currency XRP. acc: 74.30%
249/249 [==============================] - 0s 32us/step
Currency BCH. acc: 78.71%
249/249 [==============================] - 0s 32us/step
Currency LTC. acc: 75.50%
249/249 [==============================] - 0s 40us/step
Currency NEO. acc: 68.67%
249/249 [==============================] - 0s 28us/step
Currency XLM. acc: 73.09%


Testing the predictor
---

Note that 0 is when the candle is < threshold and 1 when candle is >= threshold

In [8]:
print('Prediction', np.round(models['BTC'].predict(xs)).astype(int).flatten())
print('Ref:', np.round(classes['BTC'], 1))

Prediction [0 1 0 ..., 0 0 0]
Ref: [0 1 0 ..., 0 0 0]


Testing predictor with new queried data
---

In [14]:
vars = []
for coin in currencies:
    q = get_histo(mhd, coin, block, candle_width, e)
    data = q['Data']
    var = []
    for d0, d1 in zip(data[:-1], data[1:]):
        p = d1['close']/d0['close'] - 1
        var.append(p)
    vars.append(var)
vars = np.array(vars)

for k in currencies:
    print('Prediction for currency %s' % k, np.round(models[k].predict(vars.flatten().reshape(1, -1)), 0))

Prediction for currency BTC [[ 0.]]
Prediction for currency ETH [[ 0.]]
Prediction for currency XRP [[ 0.]]
Prediction for currency BCH [[ 0.]]
Prediction for currency LTC [[ 0.]]
Prediction for currency NEO [[ 0.]]
Prediction for currency XLM [[ 0.]]
